# Weekly Report Generator
Note: this is a placeholder file; needs to be updated for Social Listening demo V3.

(I.e. the V3 Job calls this file in the correct spot, but this file won't do anything)

In [0]:
%run "./config/notebook_config"

In [0]:
%pip install openai
%pip install tabulate

In [0]:
dbutils.widgets.text("updateType", "REFRESH", "Update Type")
dbutils.widgets.text("game_name", "example game", "Game Name to Generate Report For")


In [0]:
#Set catalog
spark.sql(f"USE CATALOG {catalog_name}")

In [0]:
import pandas as pd
from datetime import datetime, timedelta
from openai import OpenAI
import os

def get_items(database_name, game, num_of_items, start_date, end_date):
    query = f"""
        SELECT translated_content_text, content_type FROM {database_name}.feedback_content_gold
        WHERE game_name = '{game}'
        AND timestamp >= '{start_date}' AND timestamp <= '{end_date}'
        LIMIT {num_of_items}
    """
    reviews_df = spark.sql(query)
    return reviews_df.pandas_api().to_markdown()

def get_total_items(database_name, game, start_date, end_date):
    query = f"""
        SELECT COUNT(*) AS total_items FROM {database_name}.feedback_content_gold
        WHERE game_name = '{game}'
        AND timestamp >= '{start_date}' AND timestamp <= '{end_date}'
    """
    df = spark.sql(query)
    return df.pandas_api().to_markdown()

def get_overall_sentiment(database_name, game, start_date, end_date):
    query = f"""
        SELECT overall_sentiment, COUNT(*) AS number_of_items
        FROM {database_name}.feedback_content_gold
        WHERE game_name = '{game}'
        AND timestamp >= '{start_date}' AND timestamp <= '{end_date}'
        GROUP BY overall_sentiment
    """
    df = spark.sql(query)
    return df.pandas_api().to_markdown()

def get_category_sentiment(database_name, game, start_date, end_date):
    query = f"""
        SELECT sentiment.category AS category_of_sentiment,
            SUM(CASE WHEN sentiment = 'positive' THEN 1 ELSE 0 END) AS number_of_positive_items,
            SUM(CASE WHEN sentiment = 'negative' THEN 1 ELSE 0 END) AS number_of_negative_items,
            SUM(CASE WHEN sentiment = 'neutral' THEN 1 ELSE 0 END) AS number_of_neutral_items
        FROM {database_name}.feedback_content_sentiment_gold AS sentiment
        INNER JOIN {database_name}.feedback_content_gold AS review ON sentiment.content_id = review.content_id
        WHERE review.game_name = '{game}'
        AND review.timestamp >= '{start_date}' AND review.timestamp <= '{end_date}'
        GROUP BY sentiment.category
    """
    df = spark.sql(query)
    return df.toPandas().to_markdown()

def get_sub_cat_sentiment(database_name, game, start_date, end_date):
    query = f"""
        SELECT category, sub_category, sentiment, count
        FROM (
            SELECT category, sub_category, sentiment, count,
                ROW_NUMBER() OVER (
                    PARTITION BY category
                    ORDER BY count DESC
                ) AS rank
            FROM (
                SELECT sentiment.category, sentiment.sub_category, sentiment.sentiment, COUNT(*) AS count
                FROM {database_name}.feedback_content_sentiment_gold AS sentiment
                INNER JOIN {database_name}.feedback_content_gold AS review ON sentiment.content_id = review.content_id
                WHERE review.game_name = '{game}'
                AND review.timestamp >= '{start_date}' AND review.timestamp <= '{end_date}'
                AND sentiment.sentiment != 'neutral'
                GROUP BY sentiment.category, sentiment.sub_category, sentiment.sentiment
            )
        )
        WHERE rank <= 5
        ORDER BY category, count DESC
    """
    df = spark.sql(query)
    return df.pandas_api().to_markdown()

def generate_report(persona, game, start_date, end_date, num_of_items, database_name, persona_prompts, llm_endpoint_name, llm_parameters):
    game = game.replace("'", "\\'")
    content_md = get_items(database_name, game, num_of_items, start_date, end_date) 
    total_items = get_total_items(database_name, game, start_date, end_date)
    overall_sentiment = get_overall_sentiment(database_name, game, start_date, end_date)
    category_sentiment = get_category_sentiment(database_name, game, start_date, end_date)
    sub_cat_sentiment = get_sub_cat_sentiment(database_name, game, start_date, end_date)

    context = (f"-Total Player Feedback Items: {str(total_items)}\n"
               f"-Overall Sentiment Distribution: {str(overall_sentiment)}\n"
               f"-Sentiment Distribution by Category: {str(category_sentiment)}\n"
               f"-Sentiment Distribution by Sub-Category: {str(sub_cat_sentiment)}\n"
               f"-The contents of {num_of_items} items: {content_md}")

    prompt = persona_prompts[persona].replace('{context}', context)

    report_contents = call_llm(llm_endpoint_name, prompt)
    
    persona_text = {"marketer": "Marketer", "game_designer": "Game Designer"}.get(persona, "Community Manager")
    subject = (f"### Subject: Review Summary Report for {persona_text} "
               f"({start_date} - {end_date})\n"
               f"**Review Summary Report for {persona_text}**\n"
               f"**Date:** {start_date} - {end_date}\n")

    return subject + report_contents




def call_llm(endpoint, prompt):
    
    DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
    client = OpenAI(
        api_key=DATABRICKS_TOKEN,
        base_url="https://e2-demo-field-eng.cloud.databricks.com/serving-endpoints"
    )

    response = client.chat.completions.create(
        model=endpoint,
        messages=[
            {
                "role": "system",
                "content": f"{prompt}"
            }
        ],
        max_tokens=5000
    )
    print(response.choices[0].message.content)
    return response.choices[0].message.content

iterate through each game and persona and run above function to generate report

In [0]:
# user steam api pull history to determine reports to generate
updateType = dbutils.widgets.get("updateType")
game_name_param = dbutils.widgets.get("game_name")

if updateType == "REFRESH":
    df_games = spark.sql(f"select game_name from {database_name}.feedback_content_gold group by game_name").pandas_api().to_pandas()
else:
    df_games = pd.DataFrame({"game_name": [game_name_param]})

df_strings = ["community_manager", "marketer", "game_designer"]
num_of_items = 10
start_date = (overall_start_date).strftime("%Y-%m-%d")
end_date = (datetime.now()).strftime("%Y-%m-%d")

reports = pd.DataFrame(
    columns=["game_name", "persona", "report_date", "report_contents"]
)

rows = []
# iterate through games and personas and generate reports
for game_name in df_games["game_name"]:
    for persona in df_strings:
        report_contents = generate_report(persona, game_name, start_date, end_date, num_of_items, database_name, persona_prompts, llm_endpoint_name, llm_parameters)
        reports = rows.append(
            {
                "game_name": game_name,
                "persona": persona,
                "report_date": pd.Timestamp.now(),
                "report_contents": report_contents,
                "start_date": start_date,
                "end_date": end_date,
            }
        )
reports = pd.DataFrame(rows)

# save reports to table
if updateType == "REFRESH":
    spark.createDataFrame(reports).write.mode("overwrite").saveAsTable(f"{database_name}.feedback_content_reports")
else:
   spark.createDataFrame(reports).write.mode("append").saveAsTable(f"{database_name}.feedback_content_reports")